In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv("Traning Datasets/DrDoS_DNS.csv").drop(columns=['Unnamed: 0',' Timestamp','SimillarHTTP'])
df.columns = df.columns.str.strip()

C:\Users\anura\AppData\Local\Temp\ipykernel_15564\3283101298.py:1: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Traning Datasets/DrDoS_DNS.csv").drop(columns=['Unnamed: 0',' Timestamp','SimillarHTTP'])


In [3]:
df.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound,Label
0,172.16.0.5-192.168.50.1-634-60495-17,172.16.0.5,634,192.168.50.1,60495,17,28415,97,0,42680.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DrDoS_DNS
1,172.16.0.5-192.168.50.1-60495-634-17,192.168.50.1,634,172.16.0.5,60495,17,2,2,0,880.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,DrDoS_DNS
2,172.16.0.5-192.168.50.1-634-46391-17,172.16.0.5,634,192.168.50.1,46391,17,48549,200,0,88000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DrDoS_DNS
3,172.16.0.5-192.168.50.1-634-11894-17,172.16.0.5,634,192.168.50.1,11894,17,48337,200,0,88000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DrDoS_DNS
4,172.16.0.5-192.168.50.1-634-27878-17,172.16.0.5,634,192.168.50.1,27878,17,32026,200,0,88000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,DrDoS_DNS


In [4]:
# Get the data types of each column
data_types = df.dtypes

In [5]:
for column in df.columns:
    col_data = df[column]
    if pd.api.types.is_numeric_dtype(col_data):
        col_range = (col_data.min(), col_data.max())
    else:
        col_range = 'N/A'  # Not applicable for non-numeric types
    print(f"Column '{column}': Data Type = {data_types[column]}, Range = {col_range}")

Column 'Flow ID': Data Type = object, Range = N/A
Column 'Source IP': Data Type = object, Range = N/A
Column 'Source Port': Data Type = int64, Range = (0, 65508)
Column 'Destination IP': Data Type = object, Range = N/A
Column 'Destination Port': Data Type = int64, Range = (0, 65535)
Column 'Protocol': Data Type = int64, Range = (0, 17)
Column 'Flow Duration': Data Type = int64, Range = (0, 119999997)
Column 'Total Fwd Packets': Data Type = int64, Range = (1, 100148)
Column 'Total Backward Packets': Data Type = int64, Range = (0, 4602)
Column 'Total Length of Fwd Packets': Data Type = float64, Range = (0.0, 15266416.0)
Column 'Total Length of Bwd Packets': Data Type = float64, Range = (0.0, 10993758.0)
Column 'Fwd Packet Length Max': Data Type = float64, Range = (0.0, 32120.0)
Column 'Fwd Packet Length Min': Data Type = float64, Range = (0.0, 2020.0)
Column 'Fwd Packet Length Mean': Data Type = float64, Range = (0.0, 3015.2905392059947)
Column 'Fwd Packet Length Std': Data Type = float6

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder();
df['Label'] = label_encoder.fit_transform(df['Label'])

In [7]:
from sklearn.model_selection import KFold
import category_encoders as ce

kf = KFold(n_splits=5, shuffle=True, random_state=42)
encoder = ce.TargetEncoder(cols=['Flow ID','Source IP','Destination IP'])
batch_size = 100000

In [8]:
def process_in_batches(train_df, test_df):
    # Iterate over the test data in chunks to reduce memory usage
    for start in range(0, len(test_df), batch_size):
        # Define end index for the current batch
        end = min(start + batch_size, len(test_df))
        
        # Extract the current batch
        batch_test_df = test_df.iloc[start:end]
        
        # Transform the batch and update the columns in place
        df.loc[batch_test_df.index, ['Flow ID', 'Source IP', 'Destination IP']] = encoder.transform(batch_test_df[['Flow ID', 'Source IP', 'Destination IP']]).values


In [9]:
# K-Fold processing
for train_index, test_index in kf.split(df):
    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]
    
    # Fit the encoder on the training data
    encoder.fit(train_df[['Flow ID', 'Source IP', 'Destination IP']], train_df['Label'])
    
    # Process the test data in batches and update the columns in place
    process_in_batches(train_df, test_df)

In [11]:
df.to_csv("Processed Datasets/DrDoS_DNS.csv" , index=False)